In [1]:
from jupyter_cadquery import set_defaults, open_viewer, show
from jupyter_cadquery.replay import replay,  enable_replay
import cadquery as cq

set_defaults(theme="dark",
            ortho=False,
            reset_camera=False)
open_viewer("CadQuery")
enable_replay(False, False)

Overwriting auto display for cadquery Workplane and Shape

Enabling jupyter_cadquery replay


In [2]:
key_sketch = cq.Sketch()
key_sketch.importDXF("kbplate.dxf")

#find the constraints of the dxf 
MIN_X =  MIN_Y = 0.0
MAX_X = MAX_Y = 0.0

KEY_W = 19.05

# for f in key_sketch.edges():
#     loc = f.Center()
#     MAX_X = max(loc.x, MAX_X)
#     # switchplate is flipped
#     MAX_Y = min(loc.y, MAX_Y)

# MAX_X += (KEY_W/2.0)
# MAX_Y -= (KEY_W/2.0)

MAX_X = KEY_W * 14
MAX_Y = KEY_W * -5

CASE_MAX_Y = MAX_Y - 3
CASE_MAX_X = MAX_X
CASE_MIN_Y = MIN_Y + 3
CASE_MIN_X = MIN_X

FILLET_RD = 1

base_sketch =(
    cq.Sketch()
    .segment((CASE_MIN_X, CASE_MIN_Y), (CASE_MAX_X, CASE_MIN_Y))
    .segment((CASE_MAX_X, CASE_MAX_Y))
    .segment((CASE_MIN_X, CASE_MAX_Y))
    .close()
    .assemble()
    .vertices()
    .fillet(FILLET_RD)
)

holes = [
        # top row
        (KEY_W,MIN_Y-2.5),
        (KEY_W*4,KEY_W*-1),
        (KEY_W*9,KEY_W*-1),
        (KEY_W*13,MIN_Y-2.5),
        #should be safe to just modify max_y
        (KEY_W,MAX_Y+2.5),
        (KEY_W*3,KEY_W*-4),
        (KEY_W*7,KEY_W*-4),
        (KEY_W*11,KEY_W*-4),
        (KEY_W*13,MAX_Y+2.5),
    ]

In [3]:
#make the case
r =(
    cq.Workplane('XY')
    .placeSketch(base_sketch)
    .extrude(1.6)
    .center(0,0)
    .placeSketch(key_sketch)
    .cutThruAll()
)
#cut the holes
plate = (
    r
    .pushPoints(holes)
    .circle(1.1)
    .tag('mounting')
    .cutThruAll()
)

show(plate)

In [4]:
#make the case
r = (
    cq.Workplane('XY')
    .placeSketch(base_sketch)
    .extrude(1.6)
)
#cut the holes
pcb = (
    r
    .pushPoints(holes)
    .circle(2)
    .tag('mounting')
    .cutThruAll()
)

show(pcb)

In [14]:
#make the base1 cutout
BLACKPILL_OFFSET = 90
BLACKPILL_X = 58
BLACKPILL_Y = 23
USBC_OFFSET = BLACKPILL_OFFSET + BLACKPILL_X
r = (
    cq.Workplane('XY')
    .placeSketch(base_sketch)
    .extrude(3)
    .center(0,0)
    #cut out the STM32 blackpill
    .sketch()
    .segment((BLACKPILL_OFFSET, CASE_MIN_Y), (BLACKPILL_OFFSET+BLACKPILL_X, CASE_MIN_Y))
    .segment((BLACKPILL_OFFSET+BLACKPILL_X, MIN_Y-BLACKPILL_Y))
    .segment((BLACKPILL_OFFSET, MIN_Y-BLACKPILL_Y))
    .close()
    .assemble()
    .finalize()
    .cutThruAll()
    .moveTo(0,0)
    #cut out the USB C
    .sketch()
    .segment((USBC_OFFSET, CASE_MIN_Y), (USBC_OFFSET, CASE_MIN_Y-9.525), tag='f')
    .segment( (USBC_OFFSET+12, CASE_MIN_Y-9.525))
    .segment((USBC_OFFSET+12, CASE_MIN_Y))
    .close()
    .assemble()
    .finalize()
    .cutThruAll()
)
#cut the holes
b1 =  (
    r.center(0,0)
    .pushPoints(holes)
    .circle(1.1)
    .tag('mounting')
    .cutThruAll()
)

show(b1)

In [15]:
#make the case
r = (
    cq.Workplane('XY')
    #base plate
    .sketch()
    .segment((CASE_MIN_X, CASE_MIN_Y), (CASE_MAX_X, CASE_MIN_Y))
    .segment((CASE_MAX_X, MIN_Y - BLACKPILL_Y))
    .segment((CASE_MIN_X, MIN_Y - BLACKPILL_Y))
    .close()
    .assemble()
    .vertices()
    .fillet(FILLET_RD)
    .finalize()
    .extrude(3)
    .center(0,0)
)
#cut the holes
b2 = (
    r.center(0,0)
    .pushPoints(holes)
    .circle(1.1)
    .tag('mounting')
    .cutThruAll()
)

show(b2)

In [17]:
#assembly stack
kb = (
    cq.Assembly()
)

kb.add(plate, name="plate", color=cq.Color("black"))
kb.add(pcb, name="pcb", color=cq.Color("black"))
kb.add(b1, name="b1", color=cq.Color("black"))
kb.add(b2, name="b2", color=cq.Color("black"))

kb.constrain("plate", "FixedRotation", (0,0,0))
kb.constrain("pcb", "FixedRotation", (0,0,0))
kb.constrain("b1", "FixedRotation", (0,0,0))
kb.constrain("b2", "FixedRotation", (0,0,0))

kb.constrain("plate@faces@<Z", "b1@faces@>Z", "Axis")
kb.constrain("plate@faces@<Z", "b1@faces@>Z", "PointInPlane", param=8)
kb.constrain("plate@faces@<Z", "pcb@faces@>Z", "Axis")
kb.constrain("plate@faces@>Z", "pcb@faces@>Z", "PointInPlane", param=5)
kb.constrain("b1@faces@<Z", "b2@faces@>Z", "Axis")
kb.constrain("b2@faces@>Z", "b1@faces@<Z",  "PointInPlane")

kb.constrain("b1", b1.edges(tag='mounting').edges(">>X[0]").first().val(),
             "b2",b2.edges(tag='mounting').edges(">>X[0]").first().val(),
             "Point", param=3)
kb.solve()

kb.save('kb.step')

This is Ipopt version 3.14.9, running with linear solver MUMPS 5.2.1.

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:       99

Total number of variables............................:       18
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  1.0336927e-02 0.00e+00 4.29e-01  -1.0 0.00e+00    -  0.00e+00 0.00e+00  